# Linear Algebra Advanced
- **Advanced decompositions**: SVD, Schur, eigenvalue problems
- **Purpose**: Extract matrix properties, dimensionality reduction, stability analysis
- **Applications**: PCA, pseudoinverse, matrix functions, spectral analysis

Key topics:
- Eigenvalue and eigenvector computation
- Singular Value Decomposition (SVD)
- Schur decomposition
- Generalized eigenvalue problems
- Matrix pseudoinverse

In [1]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

# Set print options
np.set_printoptions(precision=4, suppress=True)

print("Advanced linear algebra module loaded")

Advanced linear algebra module loaded


## Eigenvalues & Eigenvectors

For matrix \( A \), find \( \lambda \) (eigenvalue) and \( v \) (eigenvector) such that:

\[ Av = \lambda v \]

**Interpretation**:
- Eigenvector \( v \): Direction that doesn't change under transformation
- Eigenvalue \( \lambda \): Scaling factor in that direction

**Functions**:
- `eig(A)`: General matrices (complex eigenvalues)
- `eigh(A)`: Hermitian/symmetric matrices (real eigenvalues)
- `eigvals()`: Eigenvalues only (faster)

In [2]:
# Simple 2x2 matrix
A = np.array([[4, -2],
              [1,  1]])

print("Matrix A:")
print(A)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = linalg.eig(A)

print("\nEigenvalues:")
print(eigenvalues)  # May be complex

print("\nEigenvectors (columns):")
print(eigenvectors)

# Verify: A @ v = λ * v for first eigenpair
v1 = eigenvectors[:, 0]
lambda1 = eigenvalues[0]

Av1 = A @ v1
lambda_v1 = lambda1 * v1

print(f"\nVerification for first eigenpair:")
print(f"A @ v1 = {Av1}")
print(f"λ1 * v1 = {lambda_v1}")
print(f"Match: {np.allclose(Av1, lambda_v1)}")

Matrix A:
[[ 4 -2]
 [ 1  1]]

Eigenvalues:
[3.+0.j 2.+0.j]

Eigenvectors (columns):
[[0.8944 0.7071]
 [0.4472 0.7071]]

Verification for first eigenpair:
A @ v1 = [2.6833 1.3416]
λ1 * v1 = [2.6833+0.j 1.3416+0.j]
Match: True


### Symmetric/Hermitian Matrices

For **real symmetric** matrices \( A = A^T \):

**Properties**:
- All eigenvalues are **real**
- Eigenvectors are **orthogonal**
- More numerically stable computation

**Function**: `scipy.linalg.eigh(A)`

**Advantages**:
- Faster than `eig()`
- Guaranteed real eigenvalues
- Better numerical properties

In [3]:
# Create symmetric matrix
A_sym = np.array([[3, 1, 0],
                  [1, 2, 1],
                  [0, 1, 3]])

print("Symmetric matrix A:")
print(A_sym)
print(f"Is symmetric: {np.allclose(A_sym, A_sym.T)}")

# Use eigh for symmetric matrices
eigenvals, eigenvecs = linalg.eigh(A_sym)

print("\nEigenvalues (real):")
print(eigenvals)

print("\nEigenvectors:")
print(eigenvecs)

# Check orthogonality: V^T @ V = I
VTV = eigenvecs.T @ eigenvecs
print("\nV^T @ V (should be identity):")
print(VTV)
print(f"Orthogonal: {np.allclose(VTV, np.eye(3))}")

Symmetric matrix A:
[[3 1 0]
 [1 2 1]
 [0 1 3]]
Is symmetric: True

Eigenvalues (real):
[1. 3. 4.]

Eigenvectors:
[[-0.4082  0.7071  0.5774]
 [ 0.8165  0.      0.5774]
 [-0.4082 -0.7071  0.5774]]

V^T @ V (should be identity):
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0. -0.  1.]]
Orthogonal: True


In [4]:
# eigh returns eigenvalues in ascending order
# eig returns in arbitrary order

A = np.array([[5, 2],
              [2, 3]])

# Using eigh (sorted)
vals_sorted, vecs_sorted = linalg.eigh(A)
print("eigh eigenvalues (sorted):")
print(vals_sorted)

# Using eig (unsorted)
vals_unsorted, vecs_unsorted = linalg.eig(A)
print("\neig eigenvalues (unsorted):")
print(vals_unsorted.real)  # Extract real part

# Manually sort eig results
idx = vals_unsorted.real.argsort()
vals_manual_sorted = vals_unsorted[idx]
vecs_manual_sorted = vecs_unsorted[:, idx]

print("\neig eigenvalues (manually sorted):")
print(vals_manual_sorted.real)

eigh eigenvalues (sorted):
[1.7639 6.2361]

eig eigenvalues (unsorted):
[6.2361 1.7639]

eig eigenvalues (manually sorted):
[1.7639 6.2361]


### Computing Eigenvalues Only

When eigenvectors not needed:

- `eigvals(A)`: General matrices
- `eigvalsh(A)`: Symmetric/Hermitian matrices

**Faster** than computing both eigenvalues and eigenvectors

In [5]:
A = np.random.randn(4, 4)
A_sym = A + A.T  # Make symmetric

# Eigenvalues only (faster)
vals_only = linalg.eigvalsh(A_sym)
print("Eigenvalues only:")
print(vals_only)

# Compare with full decomposition
vals_full, vecs_full = linalg.eigh(A_sym)
print("\nEigenvalues from full decomposition:")
print(vals_full)

print(f"\nMatch: {np.allclose(vals_only, vals_full)}")
print("Note: eigvalsh is faster when eigenvectors not needed")

Eigenvalues only:
[-3.0791 -0.4774  1.2013  3.4375]

Eigenvalues from full decomposition:
[-3.0791 -0.4774  1.2013  3.4375]

Match: True
Note: eigvalsh is faster when eigenvectors not needed


## Singular Value Decomposition (SVD)

For any matrix \( A \) (m×n):

\[ A = U \Sigma V^T \]

Where:
- **U** (m×m): Left singular vectors (orthogonal)
- **Σ** (m×n): Diagonal matrix of singular values (σ₁ ≥ σ₂ ≥ ... ≥ 0)
- **V^T** (n×n): Right singular vectors (orthogonal)

**Function**: `scipy.linalg.svd(A, full_matrices=True)`

**Applications**:
- Pseudoinverse computation
- Matrix rank determination
- Principal Component Analysis (PCA)
- Data compression
- Least squares solutions

In [6]:
# Create a 3x2 matrix
A = np.array([[1, 2],
              [3, 4],
              [5, 6]])

print("Matrix A (3×2):")
print(A)

# Compute SVD
U, s, Vt = linalg.svd(A)

print("\nU (3×3, left singular vectors):")
print(U)

print("\nSingular values s:")
print(s)  # Returns as 1D array

print("\nV^T (2×2, right singular vectors):")
print(Vt)

# Reconstruct Σ matrix
Sigma = np.zeros((3, 2))
Sigma[:2, :2] = np.diag(s)

print("\nΣ matrix (3×2):")
print(Sigma)

# Verify: U @ Sigma @ Vt = A
A_reconstructed = U @ Sigma @ Vt
print("\nReconstructed A:")
print(A_reconstructed)
print(f"Match: {np.allclose(A, A_reconstructed)}")

Matrix A (3×2):
[[1 2]
 [3 4]
 [5 6]]

U (3×3, left singular vectors):
[[-0.2298  0.8835  0.4082]
 [-0.5247  0.2408 -0.8165]
 [-0.8196 -0.4019  0.4082]]

Singular values s:
[9.5255 0.5143]

V^T (2×2, right singular vectors):
[[-0.6196 -0.7849]
 [-0.7849  0.6196]]

Σ matrix (3×2):
[[9.5255 0.    ]
 [0.     0.5143]
 [0.     0.    ]]

Reconstructed A:
[[1. 2.]
 [3. 4.]
 [5. 6.]]
Match: True


In [7]:
A = np.array([[3, 1, 1],
              [-1, 3, 1]])

U, s, Vt = linalg.svd(A)

print("Singular values:", s)

# Property 1: Singular values are non-negative and sorted
print(f"\nNon-negative: {np.all(s >= 0)}")
print(f"Sorted descending: {np.all(s[:-1] >= s[1:])}")

# Property 2: U and V are orthogonal (U^T @ U = I)
UTU = U.T @ U
VTV = Vt.T @ Vt
print(f"\nU is orthogonal: {np.allclose(UTU, np.eye(U.shape[0]))}")
print(f"V is orthogonal: {np.allclose(VTV, np.eye(Vt.shape[1]))}")

# Property 3: Matrix rank = number of non-zero singular values
rank_svd = np.sum(s > 1e-10)  # Threshold for numerical zero
rank_actual = np.linalg.matrix_rank(A)
print(f"\nRank from SVD: {rank_svd}")
print(f"Actual rank: {rank_actual}")

Singular values: [3.4641 3.1623]

Non-negative: True
Sorted descending: True

U is orthogonal: True
V is orthogonal: True

Rank from SVD: 2
Actual rank: 2


### Reduced (Economy) SVD

For tall/wide matrices, compute only necessary components:

```python
U, s, Vt = svd(A, full_matrices=False)
```

**Full SVD** (m×n with m > n):
- U: m×m
- Σ: m×n
- V^T: n×n

**Reduced SVD** (m×n with m > n):
- U: m×n (only first n columns)
- Σ: n×n (square)
- V^T: n×n

**Benefits**: Saves memory and computation

In [8]:
# Tall matrix (more rows than columns)
A = np.random.randn(5, 3)

print(f"Matrix shape: {A.shape}")

# Full SVD
U_full, s_full, Vt_full = linalg.svd(A, full_matrices=True)
print(f"\nFull SVD:")
print(f"  U shape: {U_full.shape}")    # (5, 5)
print(f"  s shape: {s_full.shape}")    # (3,)
print(f"  Vt shape: {Vt_full.shape}")  # (3, 3)

# Reduced SVD
U_reduced, s_reduced, Vt_reduced = linalg.svd(A, full_matrices=False)
print(f"\nReduced SVD:")
print(f"  U shape: {U_reduced.shape}")    # (5, 3)
print(f"  s shape: {s_reduced.shape}")    # (3,)
print(f"  Vt shape: {Vt_reduced.shape}")  # (3, 3)

# Reconstruction is identical
A_recon = U_reduced @ np.diag(s_reduced) @ Vt_reduced
print(f"\nReconstruction match: {np.allclose(A, A_recon)}")

Matrix shape: (5, 3)

Full SVD:
  U shape: (5, 5)
  s shape: (3,)
  Vt shape: (3, 3)

Reduced SVD:
  U shape: (5, 3)
  s shape: (3,)
  Vt shape: (3, 3)

Reconstruction match: True


## Matrix Pseudoinverse

For non-square or singular matrices, compute **Moore-Penrose pseudoinverse**:

\[ A^+ = V \Sigma^+ U^T \]

Where \( \Sigma^+ \) inverts non-zero singular values: \( \sigma_i^+ = 1/\sigma_i \) if \( \sigma_i \neq 0 \)

**Function**: `scipy.linalg.pinv(A, rcond=1e-15)`

**Properties**:
- Works for any matrix (rectangular, singular)
- \( AA^+A = A \)
- \( A^+AA^+ = A^+ \)
- \( (AA^+)^T = AA^+ \)
- \( (A^+A)^T = A^+A \)

**Applications**: Least squares, rank-deficient systems

In [9]:
# Non-square matrix
A = np.array([[1, 2],
              [3, 4],
              [5, 6]])

print("Matrix A (3×2):")
print(A)

# Compute pseudoinverse
A_pinv = linalg.pinv(A)

print("\nPseudoinverse A+ (2×3):")
print(A_pinv)

# Verify properties
# Property 1: A @ A+ @ A = A
AAA = A @ A_pinv @ A
print("\nProperty A @ A+ @ A = A:")
print(f"Match: {np.allclose(AAA, A)}")

# Property 2: A+ @ A @ A+ = A+
AAA_pinv = A_pinv @ A @ A_pinv
print(f"Property A+ @ A @ A+ = A+: {np.allclose(AAA_pinv, A_pinv)}")

# Use pseudoinverse to solve least squares
b = np.array([1, 2, 3])
x_ls = A_pinv @ b
print(f"\nLeast squares solution: {x_ls}")

Matrix A (3×2):
[[1 2]
 [3 4]
 [5 6]]

Pseudoinverse A+ (2×3):
[[-1.3333 -0.3333  0.6667]
 [ 1.0833  0.3333 -0.4167]]

Property A @ A+ @ A = A:
Match: True
Property A+ @ A @ A+ = A+: True

Least squares solution: [-0.   0.5]


## Schur Decomposition

For square matrix \( A \):

\[ A = QTQ^T \]

Where:
- **Q**: Orthogonal matrix
- **T**: Upper triangular (real Schur) or quasi-triangular (complex)

**Function**: `scipy.linalg.schur(A, output='real')`

**Properties**:
- Diagonal of T contains eigenvalues
- More numerically stable than eigendecomposition
- Used in eigenvalue algorithms

**Variants**:
- `output='real'`: Real Schur form (default)
- `output='complex'`: Complex Schur form

In [10]:
# Square matrix
A = np.array([[1, 2, 3],
              [0, 4, 5],
              [0, 0, 6]])

print("Matrix A:")
print(A)

# Compute Schur decomposition
T, Q = linalg.schur(A)

print("\nSchur form T (upper triangular):")
print(T)

print("\nOrthogonal matrix Q:")
print(Q)

# Verify: A = Q @ T @ Q.T
A_reconstructed = Q @ T @ Q.T
print("\nReconstructed A:")
print(A_reconstructed)
print(f"Match: {np.allclose(A, A_reconstructed)}")

# Eigenvalues are on diagonal of T
eigenvalues_from_T = np.diag(T)
eigenvalues_direct = linalg.eigvals(A)

print("\nEigenvalues from T diagonal:", eigenvalues_from_T.real)
print("Direct eigenvalues:", np.sort(eigenvalues_direct.real))

Matrix A:
[[1 2 3]
 [0 4 5]
 [0 0 6]]

Schur form T (upper triangular):
[[1. 2. 3.]
 [0. 4. 5.]
 [0. 0. 6.]]

Orthogonal matrix Q:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Reconstructed A:
[[1. 2. 3.]
 [0. 4. 5.]
 [0. 0. 6.]]
Match: True

Eigenvalues from T diagonal: [1. 4. 6.]
Direct eigenvalues: [1. 4. 6.]


## Matrix Functions

Apply functions to matrices using eigendecomposition or other methods:

| Function | Description | Formula |
|----------|-------------|----------|
| `fractional_matrix_power()` | Matrix power \( A^p \) | Via eigendecomposition |
| `expm()` | Matrix exponential | \( e^A = \sum A^n/n! \) |
| `logm()` | Matrix logarithm | \( \log(A) \) |
| `sqrtm()` | Matrix square root | \( A^{1/2} \) |
| `sinm()`, `cosm()` | Trig functions | Matrix sine/cosine |

**Note**: These are **matrix** functions, not element-wise operations

In [11]:
# Matrix power
A = np.array([[2, 0],
              [0, 3]])

print("Matrix A:")
print(A)

# A^2 (matrix multiplication)
A_squared = A @ A
print("\nA^2 (via multiplication):")
print(A_squared)

# A^0.5 (matrix square root)
A_sqrt = linalg.fractional_matrix_power(A, 0.5)
print("\nA^0.5 (matrix square root):")
print(A_sqrt)

# Verify: (A^0.5)^2 = A
A_sqrt_squared = A_sqrt @ A_sqrt
print("\n(A^0.5)^2:")
print(A_sqrt_squared)
print(f"Match: {np.allclose(A_sqrt_squared, A)}")

# Alternative: sqrtm()
A_sqrt2 = linalg.sqrtm(A)
print("\nUsing sqrtm():")
print(A_sqrt2)
print(f"Same result: {np.allclose(A_sqrt, A_sqrt2)}")

Matrix A:
[[2 0]
 [0 3]]

A^2 (via multiplication):
[[4 0]
 [0 9]]

A^0.5 (matrix square root):
[[1.4142 0.    ]
 [0.     1.7321]]

(A^0.5)^2:
[[2. 0.]
 [0. 3.]]
Match: True

Using sqrtm():
[[1.4142 0.    ]
 [0.     1.7321]]
Same result: True


In [12]:
# Matrix exponential: e^A
A = np.array([[0, 1],
              [0, 0]])

print("Matrix A:")
print(A)

# Compute matrix exponential
expA = linalg.expm(A)
print("\ne^A (matrix exponential):")
print(expA)

# For diagonal matrices, it's element-wise exponential
D = np.diag([1, 2, 3])
expD = linalg.expm(D)
print("\nDiagonal matrix D:")
print(D)
print("\ne^D:")
print(expD)
print("\nDiagonal = [e^1, e^2, e^3]:")
print(np.diag(expD))
print("Expected:", [np.e**1, np.e**2, np.e**3])

Matrix A:
[[0 1]
 [0 0]]

e^A (matrix exponential):
[[1. 1.]
 [0. 1.]]

Diagonal matrix D:
[[1 0 0]
 [0 2 0]
 [0 0 3]]

e^D:
[[ 2.7183  0.      0.    ]
 [ 0.      7.3891  0.    ]
 [ 0.      0.     20.0855]]

Diagonal = [e^1, e^2, e^3]:
[ 2.7183  7.3891 20.0855]
Expected: [2.718281828459045, 7.3890560989306495, 20.085536923187664]


## Generalized Eigenvalue Problem

Solve:

\[ Av = \lambda Bv \]

Where both \( A \) and \( B \) are matrices.

**Function**: `scipy.linalg.eig(A, B)`

**Applications**:
- Structural dynamics (vibration analysis)
- Control theory
- Finite element analysis
- Optimization with constraints

**Special case**: When \( B = I \), reduces to standard eigenvalue problem

In [13]:
# Generalized eigenvalue problem: Av = λBv
A = np.array([[6, -2],
              [-2, 3]])
B = np.array([[2, 0],
              [0, 1]])

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

# Solve generalized eigenvalue problem
eigenvals, eigenvecs = linalg.eig(A, B)

print("\nGeneralized eigenvalues:")
print(eigenvals)

print("\nGeneralized eigenvectors:")
print(eigenvecs)

# Verify: A @ v = λ * B @ v
v1 = eigenvecs[:, 0]
lambda1 = eigenvals[0]

Av = A @ v1
lambda_Bv = lambda1 * (B @ v1)

print("\nVerification for first eigenpair:")
print(f"A @ v = {Av}")
print(f"λ * B @ v = {lambda_Bv}")
print(f"Match: {np.allclose(Av, lambda_Bv)}")

Matrix A:
[[ 6 -2]
 [-2  3]]

Matrix B:
[[2 0]
 [0 1]]

Generalized eigenvalues:
[1.5858+0.j 4.4142+0.j]

Generalized eigenvectors:
[[ 0.5774 -0.5774]
 [ 0.8165  0.8165]]

Verification for first eigenpair:
A @ v = [1.8311 1.2948]
λ * B @ v = [1.8311+0.j 1.2948+0.j]
Match: True


## Matrix Norms

Measure of matrix "size":

| Norm | Code | Definition |
|------|------|------------|
| Frobenius | `norm(A)` or `norm(A, 'fro')` | \( \sqrt{\sum_{ij} |a_{ij}|^2} \) |
| 1-norm | `norm(A, 1)` | Max column sum |
| 2-norm | `norm(A, 2)` | Largest singular value |
| ∞-norm | `norm(A, np.inf)` | Max row sum |
| Nuclear | `norm(A, 'nuc')` | Sum of singular values |

**Function**: `scipy.linalg.norm(A, ord=None)`

In [14]:
A = np.array([[1, -2, 3],
              [-1, 0, 4]])

print("Matrix A:")
print(A)

# Frobenius norm (default)
fro_norm = linalg.norm(A)
print(f"\nFrobenius norm: {fro_norm:.4f}")
print(f"  = sqrt(1² + 2² + 3² + 1² + 0² + 4²) = {np.sqrt(31):.4f}")

# 1-norm (max column sum)
norm_1 = linalg.norm(A, 1)
print(f"\n1-norm: {norm_1:.4f}")
col_sums = np.sum(np.abs(A), axis=0)
print(f"  Column sums: {col_sums}")
print(f"  Max: {np.max(col_sums):.4f}")

# Infinity norm (max row sum)
norm_inf = linalg.norm(A, np.inf)
print(f"\n∞-norm: {norm_inf:.4f}")
row_sums = np.sum(np.abs(A), axis=1)
print(f"  Row sums: {row_sums}")
print(f"  Max: {np.max(row_sums):.4f}")

# 2-norm (spectral norm = largest singular value)
norm_2 = linalg.norm(A, 2)
singular_values = linalg.svd(A, compute_uv=False)
print(f"\n2-norm (spectral): {norm_2:.4f}")
print(f"  = max singular value: {singular_values[0]:.4f}")

Matrix A:
[[ 1 -2  3]
 [-1  0  4]]

Frobenius norm: 5.5678
  = sqrt(1² + 2² + 3² + 1² + 0² + 4²) = 5.5678

1-norm: 7.0000
  Column sums: [2 2 7]
  Max: 7.0000

∞-norm: 6.0000
  Row sums: [6 5]
  Max: 6.0000

2-norm (spectral): 5.1577
  = max singular value: 5.1577


## Polar Decomposition

Every matrix can be decomposed as:

\[ A = UP \]

Where:
- **U**: Unitary/orthogonal matrix (rotation)
- **P**: Positive semidefinite Hermitian matrix (scaling)

**Function**: `scipy.linalg.polar(A)`

**Relation to SVD**: \( A = U\Sigma V^T \) gives polar as \( U_{polar} = UV^T \), \( P = V\Sigma V^T \)

In [15]:
A = np.array([[2, 1],
              [1, 2]])

print("Matrix A:")
print(A)

# Compute polar decomposition
U, P = linalg.polar(A)

print("\nUnitary matrix U (rotation):")
print(U)

print("\nPositive semidefinite P (scaling):")
print(P)

# Verify: U @ P = A
A_reconstructed = U @ P
print("\nReconstructed A = U @ P:")
print(A_reconstructed)
print(f"Match: {np.allclose(A, A_reconstructed)}")

# Check U is orthogonal
UTU = U.T @ U
print(f"\nU is orthogonal (U^T @ U = I): {np.allclose(UTU, np.eye(2))}")

# Check P is positive semidefinite (all eigenvalues >= 0)
P_eigenvals = linalg.eigvalsh(P)
print(f"\nP eigenvalues: {P_eigenvals}")
print(f"P is positive semidefinite: {np.all(P_eigenvals >= -1e-10)}")

Matrix A:
[[2 1]
 [1 2]]

Unitary matrix U (rotation):
[[ 1. -0.]
 [ 0.  1.]]

Positive semidefinite P (scaling):
[[2. 1.]
 [1. 2.]]

Reconstructed A = U @ P:
[[2. 1.]
 [1. 2.]]
Match: True

U is orthogonal (U^T @ U = I): True

P eigenvalues: [1. 3.]
P is positive semidefinite: True


## Matrix Rank and Nullspace

**Rank**: Number of linearly independent rows/columns

**Nullspace** (kernel): Set of vectors \( x \) such that \( Ax = 0 \)

**Function**: `scipy.linalg.null_space(A, rcond=None)`

**Rank-Nullity Theorem**: 
\[ \text{rank}(A) + \text{dim(null}(A)) = n \]

where \( n \) is number of columns

In [16]:
# Rank-deficient matrix
A = np.array([[1, 2, 3],
              [2, 4, 6],
              [1, 1, 1]])

print("Matrix A:")
print(A)

# Compute rank
rank = np.linalg.matrix_rank(A)
print(f"\nRank: {rank}")
print(f"Number of columns: {A.shape[1]}")

# Compute nullspace
null_space = linalg.null_space(A)
print(f"\nNullspace dimension: {null_space.shape[1]}")
print("\nNullspace basis vectors:")
print(null_space)

# Verify rank-nullity theorem
nullity = null_space.shape[1]
n = A.shape[1]
print(f"\nRank-Nullity: {rank} + {nullity} = {rank + nullity} = {n}")

# Verify: A @ v = 0 for nullspace vectors
if nullity > 0:
    v = null_space[:, 0]
    Av = A @ v
    print(f"\nVerification A @ v = {Av}")
    print(f"Close to zero: {np.allclose(Av, 0)}")

Matrix A:
[[1 2 3]
 [2 4 6]
 [1 1 1]]

Rank: 2
Number of columns: 3

Nullspace dimension: 1

Nullspace basis vectors:
[[ 0.4082]
 [-0.8165]
 [ 0.4082]]

Rank-Nullity: 2 + 1 = 3 = 3

Verification A @ v = [0. 0. 0.]
Close to zero: True


## Summary: Key Takeaways

✓ **Eigenvalues**: `eig(A)` for general, `eigh(A)` for symmetric (faster, real values)  
✓ **SVD**: `svd(A)` gives U, Σ, V^T; use `full_matrices=False` for reduced form  
✓ **Pseudoinverse**: `pinv(A)` works for any matrix; computed via SVD  
✓ **Schur decomposition**: `schur(A)` gives Q, T; eigenvalues on diagonal of T  
✓ **Matrix functions**: `expm()`, `sqrtm()`, `fractional_matrix_power()`  
✓ **Generalized eigenvalues**: `eig(A, B)` solves Av = λBv  
✓ **Matrix norms**: `norm(A, ord)` with ord='fro', 1, 2, inf, 'nuc'  
✓ **Polar decomposition**: `polar(A)` gives rotation U and scaling P  
✓ **Nullspace**: `null_space(A)` computes kernel; rank + nullity = n  

### Eigenvalue Comparison:

| Function | Use Case | Speed | Output |
|----------|----------|-------|--------|
| `eig(A)` | General matrices | Slower | Complex eigenvalues |
| `eigh(A)` | Symmetric/Hermitian | Faster | Real eigenvalues, sorted |
| `eigvals(A)` | Values only (general) | Fast | No eigenvectors |
| `eigvalsh(A)` | Values only (symmetric) | Fastest | No eigenvectors |

### Best Practices:
1. **Use eigh() for symmetric** matrices - faster and guaranteed real
2. **Reduced SVD** saves memory for tall/wide matrices
3. **Check singular values** to determine effective rank
4. **Pseudoinverse** is numerically stable for rank-deficient systems
5. **Matrix functions** via eigendecomposition are expensive for large matrices

### Next Steps:
- Explore matrix equation solvers (Sylvester, Lyapunov)
- Learn sparse eigenvalue solvers for large systems
- Study iterative methods for large-scale problems